# Welcome to the test results fucko

In [6]:
from results import Comparer, Result
import matplotlib.pyplot as plt
from os import system
from tqdm.notebook import trange

In [7]:
execute_string = "venv/bin/python3"

i_file = "-f 'instance/small case/IES_E9H12G7-v1.xlsx'"
cut_offs = [97 - 5*i for i in range(1, 19)]

args = [
    (i_file + " -c " + str(c))
    for c in cut_offs
]

execute_statements = [
    execute_string + " demo_usage.py " + arg
    for arg in args
]
execute_statements.append("venv/bin/python3 demo_usage.py -f 'instance/small case/IES_E9H12G7-v1.xlsx'")

In [8]:

l = len(execute_statements)
for i in trange(0, l):
    system(execute_statements[i])

  0%|          | 0/19 [00:00<?, ?it/s]

[05-09 14:26:24] [INFO] Parse instance data successfully.
reading instance runs for 0.084s
forming PTDF matrix runs for 0.001s
forming LODF matrix runs for 0.000s
[05-09 14:26:24] [INFO] remove 6/51 nonzero elements in PTDF that are less than 1e-6.
[05-09 14:26:24] [INFO] remove 8/39 nonzero elements in LODF that are less than 1e-6.
initializing the electricity network runs for 0.003s
initializing the gas network runs for 0.054s
initializing the heating network runs for 0.060s
Set parameter Username
Academic license - for non-commercial use only - expires 2024-04-18
Set parameter NumericFocus to value 3
Warning for adding constraints: zero or small (< 1e-13) coefficients, ignored
add security constraints of sources (pre-calculation) runs for 0.240s
Gurobi Optimizer version 10.0.1 build v10.0.1rc0 (linux64)

CPU model: 12th Gen Intel(R) Core(TM) i5-12500H, instruction set [SSE2|AVX|AVX2]
Thread count: 16 physical cores, 16 logical processors, using up to 16 threads

Optimize a model wit